In [ ]:
#imports for model 

from mast3r.model import AsymmetricMASt3R

#general imports for plotting and visualization 
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from PIL import Image


#my utilities for evaluation and general dataset reading
from utils import CameraMatrix, computePoseError
from myDataset import ImagePairDataset, ResultsDataset, readResultsFile
from sevenScenesDatasets import loadPose7scenes

#load model
device = 'cuda:4'
model_name = "naver/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric"
model = AsymmetricMASt3R.from_pretrained(model_name).to(device)

#model hyperprams
K = CameraMatrix(585,585,320,240) #specific to dataset
n_matches = 30





In [ ]:
#reading the pairs file

from sevenScenesDatasets import readMultiImageRelPoseNetPairsFile #function will change per pair file
root_dir = "/datasets/7scenes_org"
pairs_file = "/home/bjangley/VPR/7scenes/pairs2/test_tuples_multiimagerelposenet.txt"

anchors, queries, scene_ids = readMultiImageRelPoseNetPairsFile(pairs_file, root_dir)
dataset = ImagePairDataset(anchors, queries)
# print(len(dataset))
# print(dataset[0])



In [ ]:
#mast3r output -- might need changes -- base case will just find query to anchor transformation 
#for 7scenes specifically, i make use of the ground truth anchor to world transformation

from tqdm import tqdm
from myMethod import processPair

# output_file = 'pairResults.txt'

# # Get the total number of items to process
# total_items = len(dataset)

# with open(output_file, 'w') as f:
#     # Create a tqdm progress bar
#     for idx in tqdm(range(total_items), desc="Processing pairs", unit="pair"):
#         item = dataset[idx]
#         anchor_path = item['anchor_path']
#         query_path = item['query_path']
#         anchor_pose = loadPose7scenes(anchor_path) #load pose for anchor 

#         ret_val, transformation, n_matches_total, n_matches_filtered = processPair(
#             model, device, anchor_path, query_path, K, n_matches
#         )
#         # Write output
#         f.write(f"{query_path} {anchor_path} {n_matches_total} {n_matches_filtered} {ret_val}")
#         if ret_val:
#             mast3r_query2world = np.dot(anchor_pose, transformation)
#             f.write(" " + " ".join(map(str, transformation.flatten())))
#             f.write(" " + " ".join(map(str, mast3r_query2world.flatten())))
#         else:
#             f.write(" " + " ".join(["0"] * 32)) #return an array of zeros if fail
#         f.write("\n")
#         f.flush()

In [ ]:
import numpy as np

# Usage
output_file = '/home/bjangley/VPR/mast3r/results_n30_withlogs.txt'
results = readResultsFile(output_file)


results.printSummary() # Print summary

## you can access an individual element by index using item = results.getPairResults(i)
## then access further information with item[key] with key being 'n_matches_total', 'n_matches_filtered', 'ret_val','mast3r_q2a','mast3r_q2world'
## the results class also has additional functions that return a list of indices 
#### lists returned::
#### getMatchesBelow(threshold)
#### getMatchesWithin(lower,Upper)
#### getFails() --> returns indices of all fails -- transform = 0

In [ ]:

indices = results.getMatchesWithin(1000,2000)
dataset.visualizePairs(indices)



